# Problem Set 2

In [6]:
using Random, Distributions
using Plots
#using PlotlyJS
using StatsPlots
pyplot()

LoadError: ArgumentError: Package PyPlot not found in current path.
- Run `import Pkg; Pkg.add("PyPlot")` to install the PyPlot package.

In [2]:
gaussian = Normal(2,3)

Normal{Float64}(μ=2.0, σ=3.0)

In [3]:
seq = rand(gaussian, 2000)

2000-element Vector{Float64}:
 -0.06259995045072575
  2.628989989212921
  1.282703089070524
  3.8073376330645363
 -2.1361314113794574
 -4.784560371721618
  4.09834539740636
  3.1595911392282305
  3.8670326978805227
  2.106710316022436
  6.0156172988056875
  1.9408756359128547
 -1.4690297290159293
  ⋮
  3.205264306213961
 -3.889683114969248
  1.2313697934316128
  4.124044971183686
  5.138547940239389
  0.3586404681080899
  5.168802595785193
 -1.7981109278038345
  3.0797581183211347
 -0.5140023675377292
 -1.4623903203638688
 -0.46476545314049744

In [40]:
transpose(seq)

1×2000 transpose(::Vector{Float64}) with eltype Float64:
 2.901  12.1718  1.0243  4.7067  0.862809  …  0.0983106  5.65109  -0.34041

In [53]:
histogram(rand(gaussian,2000), nbinsx = 35)

In [5]:
PlotlyJS.plot(seq,x = :number,kind="histogram",nbinsx = 35, histnorm = "probability density")

data: [
  "histogram with fields histnorm, nbinsx, type, x, and y"
]

layout: "layout with fields margin and template"

In [4]:
plot(seq, seriestype = :histogram)